In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [37]:
from scipy.special import softmax
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.optimize import LinearConstraint, Bounds, minimize
import warnings
warnings.filterwarnings("ignore", message="delta_grad == 0.0. Check if the approximated function is linear.")

class SchoolManifold():

    def __init__(self, embeddings_df: pd.DataFrame, k = 15, ):
        self.embeddings_df = embeddings_df
        self.embeddings = self.embeddings_df.drop(labels = ["unitid"], axis =1)

        self.k = k

    
    def _compute_neighbors(self, k: int):
        knn = NearestNeighbors(n_neighbors = k, algorithm="auto")
        knn.fit(self.embeddings.values)
        dists, indices = knn.kneighbors(self.embeddings.values)
        self.neighbor_dists = dists
        self.neighbor_indices = indices


    def preprocess_for_target(self, target_df: pd.DataFrame, target_col_ind: int = None, target_col_lab = None, total_column_lab = None):
        if (target_col_ind is not None):
            target_col_lab = target_df.columns[target_col_ind]

        if (target_col_lab is None):
            raise ValueError("Either target_col_ind or target_col_lab must not be None")

        self.total_column_lab = total_column_lab
        merged = pd.merge(self.embeddings_df, target_df, how= "inner", on = "unitid")
        
        self.embeddings = merged[self.embeddings.columns]
        self._compute_neighbors(self.k)

        self.target_df = merged[target_df.columns]
        self.features = self.target_df.drop(labels = ["unitid", target_col_lab], axis = 1)
        orig_columns = self.features.columns
        self.target = self.target_df[target_col_lab]
        self.schools = self.target_df["unitid"]

        inds_of_target_in_original = merged.index.values
        neighbor_inds_of_target_in_original = self.neighbor_indices[inds_of_target_in_original]
        neighbor_dists_of_target_col = self.neighbor_dists[inds_of_target_in_original]

        vals_of_target_col = merged[target_col_lab].values
        neighbor_vals_of_target_col =  vals_of_target_col[neighbor_inds_of_target_in_original]
        
        features_of_target_col = merged[orig_columns].values
        neighbor_features_of_target_col = features_of_target_col[neighbor_inds_of_target_in_original]

        mask = neighbor_vals_of_target_col > 1.1 * vals_of_target_col.reshape(-1, 1)
        neighbor_weights_of_target_col = np.where(mask, neighbor_dists_of_target_col, np.PINF)
        neighbor_weights_of_target_col = softmax(-neighbor_weights_of_target_col, axis = 1)

        self.mask = mask
        self.v_star = (neighbor_vals_of_target_col * neighbor_weights_of_target_col).sum(axis = 1)
        self.weights =  neighbor_weights_of_target_col
        self.neighbor_features = neighbor_features_of_target_col
        self.neighbor_target = neighbor_vals_of_target_col
        
        if (total_column_lab is not None):
            self.total_column = self.features[total_column_lab].values.reshape(-1,1)
            self.total_column_index = list(self.features.columns).index(total_column_lab)
            self.neighbor_features[:, :, self.total_column_index] = self.total_column
        
        self.vstars = np.where(np.isnan(self.v_star), self.target.values, self.v_star)
    

    def compute_deltas(self, reg, mode = "projection", eps = 0.1):
        self.reg = reg
        self.adjusted_features = np.sum(self.neighbor_features * self.weights[:, :, None], axis = 1)
        self.adjusted_features = np.where(np.isnan(self.adjusted_features), self.features.values, self.adjusted_features)

        if mode == "optimize":
            def objective_xgb(x, reg, df_cols):
                x_df = pd.DataFrame(x.reshape(1, -1), columns=list(df_cols))
                x = xgb.DMatrix(x_df)
                v = reg.predict(x)
                return -v
            
            def objective(x, reg, df_cols):
                v = reg.predict(pd.DataFrame(x.reshape(1,-1), columns=list(df_cols)))
                return -v
            d = self.adjusted_features.shape[-1]
            self.vhats = []
            
            for school, x0 in enumerate(self.features.values):
                if (np.allclose(x0, self.adjusted_features[school])):
                    self.vhats.append(self.target[school])


                vstar = self.vstars[school]

                if (self.total_column_lab is not None):
                    ub = np.zeros(d)
                    ub[self.total_column_index] = x0[self.total_column_index]
                    other_cols = [x for x in range(d) if x != self.total_column_index]
                    ub[other_cols] = x0[other_cols] + eps * x0[other_cols]
                    
                    bounds = Bounds(lb = np.zeros(d), ub = ub, keep_feasible=True)
                else:
                    bounds = Bounds(lb = np.zeros(d), ub = x0.flatten() + eps*x0.flatten(), keep_feasible=True)
                
                if type(reg) == xgb.core.Booster:
                    res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)
                else:
                    res = minimize(objective, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)

                self.vhats.append(-res.fun)
                self.adjusted_features[school] = res.x
                print(f"vhat: {-res.fun}, vstar: {vstar}, orig: {self.target[school]}")
             
                
        outcome_results = np.zeros((len(self.vhats), 3))
        outcome_results[:, 0] = self.target[:len(self.vhats)]
        outcome_results[:, 1] = self.vhats[:len(self.vhats)]
        outcome_results[:, 2] = self.vstars[:len(self.vhats)]
        
        outcome_results_df = pd.DataFrame(outcome_results, columns = ["original", "vhat", "vstar"])
        self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)

        self.results = pd.concat((outcome_results_df, self.adjusted_features_df), axis = 1)
        self.deltas = self.adjusted_features - self.features.values

        return self.results

        

### Graduation Rate

In [38]:
ic_mca = pd.read_csv("../datasets/Clusters/IC_CLUSTERS_AND_MCA.csv")
ic_mca = ic_mca.loc[:, ic_mca.columns != "ic_cluster"]

grad_rates = pd.read_csv("../datasets/Target Features/grad_rate_per_dollar.csv", index_col=0)

In [39]:
manifold = SchoolManifold(ic_mca)

In [40]:
manifold.preprocess_for_target(grad_rates, target_col_lab="grtotlt", total_column_lab=None)

In [41]:
import xgboost as xgb

xgb_reg = xgb.Booster()
xgb_reg.load_model("../models/grad_rate_per_dollar.json")

In [42]:
results = manifold.compute_deltas(xgb_reg, mode = "optimize")

vhat: -17.34075355529785, vstar: -19.32711942593674, orig: -20.505650507882223
vhat: -18.67205047607422, vstar: -19.224629007855608, orig: -19.03401105419688
vhat: -17.613388061523438, vstar: -17.995984207202703, orig: -18.711707647609444
vhat: -20.339717864990234, vstar: -19.982776254322363, orig: -20.406578489708583
vhat: -17.994182586669922, vstar: -19.54042260167883, orig: -19.40800899814603
vhat: -18.755355834960938, vstar: -18.948270715517776, orig: -18.584419776769877
vhat: -17.07676124572754, vstar: -17.852580608392746, orig: -17.912952386542134
vhat: -17.593414306640625, vstar: -18.478650123613576, orig: -18.081279852425236
vhat: -19.03741455078125, vstar: -18.825429378026776, orig: -18.93721378191786
vhat: -19.271371841430664, vstar: -19.28768325217141, orig: -19.413311531637287
vhat: -20.76112937927246, vstar: -20.684901478178908, orig: -20.82567820801087
vhat: -17.010892868041992, vstar: -17.40660477836902, orig: -16.509857705627137
vhat: -18.634212493896484, vstar: -19.754

In [ ]:
results.to_csv("../results/grad_rate_results.csv", index = False)

0.5165707657896967

### Plots 

In [ ]:
results = pd.read_csv("../results/grad_rate_results.csv")